#Feature Extraction(skip if .csv made)

In [56]:
import librosa
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

In [13]:
import pandas as pd

fp = pd.read_csv("file_path_raaga.csv")
fp.head()

,Unnamed: 0,filepath,mbid,raagas
0,13,Ashwath Narayanan at Arkay by Ashwath Narayana...,ffe5da02-8a71-4933-b414-62ca298dc777,purna chandrika
1,14,Ashwath Narayanan at Arkay by Ashwath Narayana...,a724bc1d-0eaa-44d1-ae2a-401c6ae62865,surati
2,15,Ashwath Narayanan at Arkay by Ashwath Narayana...,455c858a-da98-4094-a41d-41d0154037f2,shuddha saveri
3,16,Ashwath Narayanan at Arkay by Ashwath Narayana...,662dbd9a-6858-4899-9369-8079c6c22935,mohanam
4,17,Ashwath Narayanan at Arkay by Ashwath Narayana...,ab8fff58-8359-4af8-bc2a-6d623d4ac48b,thodi


In [60]:
from tqdm import tqdm  # Import tqdm for progress tracking

# Define the columns of the dataset DataFrame
columns = ['rmse', 'chroma_stft', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'raga']
dataset = pd.DataFrame(columns=columns)
ragas = fp["raagas"].unique()

# Iterate over each raga and sample
for raga in tqdm(ragas, desc="Processing ragas"):
    sample = fp[fp["raagas"] == raga]
    
    # Iterate over each song in the sample
    for song in tqdm(sample["filepath"], desc="Processing songs", leave=False):
        sr = 0
        songname = f"D:/carnatic/{song}.mp3.mp3"
        y, sr = librosa.load(songname, mono=True)
        dur = librosa.get_duration(y=y, sr=sr)
        off = 0

        # Check duration and process accordingly
        if dur > 300:
            for i in tqdm(range(10), desc="Processing segments", leave=False):
                x, sr = librosa.load(songname, mono=True, offset=off, duration=30)
                rmse = librosa.feature.rms(y=x)[0]
                chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
                rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
                zcr = librosa.feature.zero_crossing_rate(x)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13)
                
                # Create data list and series
                data = [
                    np.mean(rmse), np.mean(chroma_stft), np.mean(spec_cent),
                    np.mean(spec_bw), np.mean(rolloff), np.mean(zcr),
                    np.mean(mfcc[0]), np.mean(mfcc[1]), np.mean(mfcc[2]),
                    np.mean(mfcc[3]), np.mean(mfcc[4]), np.mean(mfcc[5]),
                    np.mean(mfcc[6]), np.mean(mfcc[7]), np.mean(mfcc[8]),
                    np.mean(mfcc[9]), np.mean(mfcc[10]), np.mean(mfcc[11]),
                    np.mean(mfcc[12]), raga
                ]
                dataseries = pd.Series(data, index=dataset.columns)
                dataset = dataset._append(dataseries, ignore_index=True)

                # Update offset
                if i in range(0, 3):
                    off += 30
                if i in range(3, 7):
                    off = (dur / 10) * i
                if i in range(7, 10):
                    off = dur - ((10 - i) * 30)

        else:
            for i in tqdm(range(10), desc="Processing segments", leave=False):
                x, sr = librosa.load(songname, mono=True, offset=off, duration=(dur / 10))
                rmse = librosa.feature.rms(y=x)[0]
                chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
                rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
                zcr = librosa.feature.zero_crossing_rate(x)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13)
                
                # Create data list and series
                data = [
                    np.mean(rmse), np.mean(chroma_stft), np.mean(spec_cent),
                    np.mean(spec_bw), np.mean(rolloff), np.mean(zcr),
                    np.mean(mfcc[0]), np.mean(mfcc[1]), np.mean(mfcc[2]),
                    np.mean(mfcc[3]), np.mean(mfcc[4]), np.mean(mfcc[5]),
                    np.mean(mfcc[6]), np.mean(mfcc[7]), np.mean(mfcc[8]),
                    np.mean(mfcc[9]), np.mean(mfcc[10]), np.mean(mfcc[11]),
                    np.mean(mfcc[12]), raga
                ]
                dataseries = pd.Series(data, index=dataset.columns)
                dataset = dataset._append(dataseries, ignore_index=True)
                
                # Update offset
                off = (dur / 10) * i


Processing songs:   0%|                                                                          | 0/2 [00:00<?, ?it/s]

Processing segments:   0%|                                                                      | 0/10 [00:00<?, ?it/s]C:\Users\Rajeev Sekar\AppData\Local\Temp\ipykernel_26524\3558046319.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = dataset._append(dataseries, ignore_index=True)


Processing segments:  10%|██████▏                                                       | 1/10 [00:00<00:02,  4.07it/s]

Processing segments:  20%|████████████▍                                                 | 2/10 [00:00<00:01,  5.10it/s]

Processing segments:  30%|██████████████████▌                                           

KeyboardInterrupt: 

In [57]:
data = pd.read_csv("dataset.csv")
data.shape

(1840, 21)

In [43]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,raga
0,0.031511,0.317603,1708.003423,2073.255115,3612.529119,0.060936,-322.84380,109.459850,-2.044547,7.518318,-11.074583,5.313380,-22.433044,-11.289671,-8.702506,-8.501856,-9.121349,-2.449628,-11.605868,purna chandrika
1,0.031511,0.317603,1708.003423,2073.255115,3612.529119,0.060936,-322.84380,109.459850,-2.044547,7.518318,-11.074583,5.313380,-22.433044,-11.289671,-8.702506,-8.501856,-9.121349,-2.449628,-11.605868,purna chandrika
2,0.044276,0.338240,1690.241139,2038.245137,3691.999088,0.060042,-244.12134,118.934510,-5.504768,12.037164,-20.277803,9.172882,-22.398329,-12.889206,-6.194196,-10.408050,-11.628651,0.368444,-13.271333,purna chandrika
3,0.033655,0.283715,1707.162938,2062.905089,3553.485722,0.063138,-298.48642,111.781050,0.330065,12.399558,-15.397969,5.719809,-20.228254,-10.876355,-11.663799,-9.163312,-11.074248,-1.787798,-10.519550,purna chandrika
4,0.051188,0.287830,2224.945233,2284.799850,4762.157785,0.080556,-224.02823,87.893936,-7.011757,16.922638,-22.737362,11.527616,-34.319733,-6.622852,-3.180578,-10.729729,-11.729633,4.459792,-18.696102,purna chandrika


#Model Training

In [58]:
import os
import librosa
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [59]:
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
Y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

#ANN Model

In [18]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(96, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

D:\Python\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 256)                 │           5,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 96)                  │           6,240 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,512 (205.12 KB)

 Trainable params: 52,512 (205.12 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
classifier = model.fit(X_train, y_train,batch_size=128, epochs=500, validation_data=(X_test,y_test),callbacks=[checkpoint])

Epoch 1/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 8s 820ms/step - accuracy: 0.0078 - loss: 4.6432
Epoch 1: val_loss improved from inf to 4.47756, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0230 - loss: 4.5657 - val_accuracy: 0.0304 - val_loss: 4.4776
Epoch 2/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0625 - loss: 4.3990
Epoch 2: val_loss improved from 4.47756 to 4.39939, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0562 - loss: 4.4108 - val_accuracy: 0.0348 - val_loss: 4.3994
Epoch 3/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0625 - loss: 4.2563
Epoch 3: val_loss improved from 4.39939 to 4.32503, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0598 - loss: 4.3180 - val_accuracy: 0.0500 - val_loss: 4.3250
Epoch 4/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0859 - loss: 4.2753
Epoch 4: val_loss improved from 4.32503 to 4.25640, saving

In [20]:
y_pred = model.predict(X)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Total accuracy on the entire dataset (X and Y): 0.8755


LSTM Model

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
model = Sequential()
adam1 = Adam(learning_rate=0.01)
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.25, return_sequences=True,input_shape=input_shape))
model.add(LSTM(units=64,  dropout=0.05, recurrent_dropout=0.25, return_sequences=True))
model.add(Flatten())
model.add(Dense(units=96, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam1, metrics=["accuracy"],)
model.summary()

D:\Python\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 1, 128)              │          75,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 1, 64)               │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 96)                  │           6,240 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 131,424 (513.38 KB)

 Trainable params: 131,424 (513.38 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
batch_size = 35 # num of training examples per minibatch
num_epochs =300
classify2 = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_test,y_test),
)

Epoch 1/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.0561 - loss: 4.3802 - val_accuracy: 0.0978 - val_loss: 3.8075
Epoch 2/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1113 - loss: 3.4193 - val_accuracy: 0.1500 - val_loss: 3.1760
Epoch 3/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2339 - loss: 2.6838 - val_accuracy: 0.1804 - val_loss: 2.8563
Epoch 4/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2719 - loss: 2.4248 - val_accuracy: 0.2174 - val_loss: 2.5919
Epoch 5/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3845 - loss: 2.0545 - val_accuracy: 0.3065 - val_loss: 2.3746
Epoch 6/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4209 - loss: 1.8605 - val_accuracy: 0.3478 - val_loss: 2.2226
Epoch 7/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4312 - loss: 1.7047 - val_accuracy: 0.3565 - val_loss: 2.1400
Epoch 8/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5142 - loss: 1.5558 - val_accuracy: 0.3674 - 

In [28]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Total accuracy on the entire dataset (X and Y): 0.8707


#BiLSTM Model

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
model = Sequential()
adam1 = Adam(learning_rate=0.001)
model.add(Bidirectional(LSTM(units=128, dropout=0.05, recurrent_dropout=0.25, return_sequences=True),input_shape=input_shape))
model.add(Bidirectional(LSTM(units=64,  dropout=0.05, recurrent_dropout=0.25, return_sequences=True)))
model.add(Flatten())
model.add(Dense(units=96, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam1, metrics=["accuracy"],)
model.summary()

D:\Python\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional)      │ (None, 1, 256)              │         151,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 1, 128)              │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 96)                  │          12,384 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 328,288 (1.25 MB)

 Trainable params: 328,288 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
batch_size = 35 # num of training examples per minibatch
num_epochs = 500
classify = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_test,y_test),
)

Epoch 1/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.0464 - loss: 4.5477 - val_accuracy: 0.0978 - val_loss: 4.4602
Epoch 2/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1058 - loss: 4.3782 - val_accuracy: 0.1022 - val_loss: 4.1044
Epoch 3/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1040 - loss: 3.9892 - val_accuracy: 0.1065 - val_loss: 3.7849
Epoch 4/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1145 - loss: 3.6021 - val_accuracy: 0.1174 - val_loss: 3.5175
Epoch 5/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1619 - loss: 3.2386 - val_accuracy: 0.1435 - val_loss: 3.2665
Epoch 6/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2022 - loss: 2.9775 - val_accuracy: 0.1739 - val_loss: 3.0646
Epoch 7/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2646 - loss: 2.7461 - val_accuracy: 0.2000 - val_loss: 2.9009
Epoch 8/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2831 - loss: 2.5628 - val_accuracy: 0.2087 - 

In [32]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Total accuracy on the entire dataset (X and Y): 0.8783


# 1D Convolution Network

In [60]:
import os
import librosa
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
import tensorflow.keras.layers as layers
import keras
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [62]:
data = pd.read_csv('dataset.csv')
# Dropping unneccesary columns
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
Y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
X_train.shape[1], X_train.shape[2]

(1, 20)

In [63]:
model = tf.keras.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape))
model.add(layers.Conv1D(128, 3, activation='relu', padding='same'))
model.add(layers.Conv1D(256, 3, activation='sigmoid', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(96, activation='softmax'))
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

D:\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)                   │ (None, 1, 64)               │           3,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_15 (Conv1D)                   │ (None, 1, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_16 (Conv1D)                   │ (None, 1, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 96)                  │          12,384 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 172,448 (673.62 KB)

 Trainable params: 172,448 (673.62 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
classifier = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=1000,
    validation_data=(X_test, y_test),
)

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0111 - loss: 4.6576 - val_accuracy: 0.0471 - val_loss: 4.4422
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0373 - loss: 4.4249 - val_accuracy: 0.0743 - val_loss: 4.4028
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0602 - loss: 4.3900 - val_accuracy: 0.0507 - val_loss: 4.3936
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0550 - loss: 4.3624 - val_accuracy: 0.0344 - val_loss: 4.3794
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0322 - loss: 4.3521 - val_accuracy: 0.0707 - val_loss: 4.3517
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0428 - loss: 4.3097 - val_accuracy: 0.0598 - val_loss: 4.3052
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0425 - loss: 4.2632 - val_accuracy: 0.0562 - val_loss: 4.2484
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0585 - loss: 4.2043 - val_accuracy: 0

In [ ]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(Y, y_pred_classes)


print(f"accuracy: {accuracy:.4f}")